## Setup Only for Colab

In [ ]:
import scipy.stats
np.round(scipy.stats.chi2(1 + 1).ppf(1 - .05), 5)


In [ ]:
from proximalde.proximal import estimate_nuisances
def gen_data_no_controls(n, pw, pz, px, a, b, c, d, e, f, g, *, sm=2, sz=1, sx=1, sy=1):
    ''' Controls are generated but are irrelevant to the rest
    of the data

    n: number of samples
    pw: dimension of controls
    pz: dimension of treatment proxies ("instruments")
    px: dimension of outcome proxies ("treatments")
    a : strength of D -> M edge
    b : strength of M -> Y edge
    c : strength of D -> Y edge
    d : strength of D -> Z edge
    e : strength of M -> Z edge
    f : strength of M -> X edge
    g : strength of X -> Y edge
    sm : scale of noise of M
    sz : scale of noise of Z
    sx : scale of noise of X
    sy : scale of noise of Y
    '''
    W = None#np.random.normal(0, 1, size=(n, pw))
    D = np.random.binomial(1, .5 * np.ones(n,))
    M = a * D + sm * np.random.normal(0, 1, (n,))
    Z = (e * M + d * D).reshape(-1, 1) + sz * np.random.normal(0, 1, (n, pz))
    X = f * M.reshape(-1, 1) + sx * np.random.normal(0, 1, (n, px))
    Y = b * M + c * D + g * X[:, 0] + sy * np.random.normal(0, 1, n)
    return W, D, M, Z, X, Y
n=10000
np.random.seed(123)
W, D, _, Z, X, Y = gen_data_no_controls(n, pw=1, pz=1, px=1, a=1., b=1., c=.5, d=0.0, e=0.0, f=1.0, g=0.0)
D = D - D.mean(axis=0, keepdims=True)
Z = Z - Z.mean(axis=0, keepdims=True)
X = X - X.mean(axis=0, keepdims=True)
_, _, _, _, _, _, pval, dval, strength, strength_std, *_ = estimate_nuisances(D, Z, X, Y,
                                                                                cv=5, n_jobs=-1, verbose=0,
                                                                                random_state=123)
print(pval, dval, strength / strength_std)
assert dval > 3.84
assert pval < 3.84
assert strength / strength_std > 11.0

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/hidden_mediators

In [ ]:
%ls

In [ ]:
from IPython.display import clear_output

In [ ]:
import time
!pip install -r requirements.txt
time.sleep(2)
clear_output()

In [ ]:
import time
# replace `develop` with `install` if you wont make library code changes
!python setup.py develop
time.sleep(2)
clear_output()
# Restart the session after running this

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

# Main Logic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import pytest
from joblib import Parallel, delayed
from proximalde.gen_synthetic_data import gen_data_no_controls
from proximalde.proximal import ProximalDE

In [ ]:
def exp_summary(it, n, pz, px, a, b, c, d, e, f, g, sm):
    np.random.seed(it)
    _, D, _, Z, X, Y = gen_data_no_controls(n, pz, px, a, b, c, d, e, f, g, sm=sm)
    est = ProximalDE(cv=3, semi=True, ivreg_type='adv', n_jobs=1, random_state=3, verbose=0)
    est.fit(None, D, Z, X, Y)
    lb, ub = est.robust_conf_int(lb=-2, ub=2)
    weakiv_stat, _, _, _, pi, var_pi = est.weakiv_test(return_pi_and_var=True)
    eigs, eig_crit = est.covariance_rank_test(calculate_critical=True)
    maxeig = eigs[0]
    return est.stderr_, est.idstrength_, est.primal_violation_, est.dual_violation_, est.point_, \
        lb, ub, weakiv_stat, maxeig, pi, var_pi, *est.gamma_.flatten(), \
        *est.ivreg_gamma_.stderr_.flatten(), est.idstrength_std_, eig_crit

def run_summary(n, pz, px, a, b, c, d, e, f, g, sm):
    res = np.array(Parallel(n_jobs=-1, verbose=3)(delayed(exp_summary)(it, n, pz, px, a, b, c, d, e, f, g, sm)
                                                  for it in range(100)))
    print(f"Mean estimate: {np.mean(res[:, 4]):.3f}")
    print(f"Bias: {np.mean(res[:, 4] - c):.3f}")
    print(f"RMSE: {np.sqrt(np.mean((res[:, 4] - c)**2)):.3f}")
    cov = np.mean((res[:, 4] + 1.96 * res[:, 0] >= c) & (res[:, 4] - 1.96 * res[:, 0] <= c))
    print(f"Coverage: {cov:.3f}")
    rcov = np.mean((res[:, 5] <= c) & (res[:, 6] >= c))
    print(f"ID-Robust Coverage: {rcov:.3f}")
    plt.figure(figsize=(10, 10))
    plt.subplot(3, 2, 1)
    plt.title(f"stderr: mean={np.mean(res[:, 0]):.3f}\n%>1.0={np.mean(res[:, 0] > 1.0):.3f}")
    plt.hist(res[:, 0])
    plt.subplot(3, 2, 2)
    crit = np.round(scipy.stats.foldnorm(c=10).ppf(.95), 2)
    plt.title(f"idstrength: mean={np.mean(res[:, 1] / res[:, 11 + 2*pz]):.3f}\n%Fail (<{crit})={np.mean(res[:, 1] / res[:, 11 + 2*pz] < crit):.3f}")
    plt.hist(res[:, 1] / res[:, 11 + 2*pz])
    plt.subplot(3, 2, 3)
    crit = np.round(scipy.stats.chi2(df=pz + 1).ppf(.95), 2)
    plt.title(f"primal_violation: mean={np.mean(res[:, 2]):.3f}\n%Fail (>{crit})={np.mean(res[:, 2] > crit):.3f}")
    plt.hist(res[:, 2])
    plt.subplot(3, 2, 4)
    crit = np.round(scipy.stats.chi2(df=px).ppf(.95), 2)
    plt.title(f"dual_violation: mean={np.mean(res[:, 3]):.3f}\n%Fail (>{crit})={np.mean(res[:, 3] > crit):.3f}")
    plt.hist(res[:, 3])
    plt.subplot(3, 2, 5)
    plt.title(f"weakiv_test: mean={np.mean(res[:, 7]):.3f}\n%Fail (<23)={np.mean(res[:, 7] < 23):.3f}")
    plt.hist(res[:, 7])
    plt.subplot(3, 2, 6)
    eig_stat = res[:, 8] / res[:, 11+2*pz+1]
    plt.title(f"rank_test: mean={np.mean(eig_stat):.3f}\n%Fail (<1)={np.mean(eig_stat < 1):.3f}")
    plt.hist(eig_stat)
    plt.tight_layout()
    plt.show()


```
a : strength of D -> M edge
b : strength of M -> Y edge
c : strength of D -> Y edge
d : strength of D -> Z edge
e : strength of M -> Z edge
f : strength of M -> X edge
g : strength of X -> Y edge
```

## No Failure Mode

In [ ]:
np.random.seed(1)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has direct relationship to M,
# X has direct relationship to M, X has direct relationship to Y (but doesn't matter w/ validity of result)
d, e, f, g = 1.0, 1.0, 1.0, 1.0
sm = 2.0
run_summary(n, pz, px, a, b, c, d, e, f, g, sm)

## Failure mode 1: Z is a bad proxy (no M->Z)

Z has a direct relationship to D but no direct relationship to M. So Z is not a good proxy treatment. However, X is a good proxy outcome and this can allow us to detect the failure mode. 

Here the fact that Z is correlated with D makes the dual assumption not be violated, and the primal should pass as X is a good proxy outcome. However, the solution basically leads to an orthogonal instrument V=D-gammaZ that perfectly predicts treatment D (since Z is only driven by D) and hence the identification strength assumption is violated. Thus we expect the id_stregth test should fail.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has no relationship to M,
# X has direct relationship to M, X has direct relationship to Y (but doesn't matter w/ validity of result)
d, e, f, g = 1, 0, 1, 0
sm = 2.
n = 10000
px, pz = 5, 5
run_summary(n, pz, px, a, b, c, d, e, f, g, sm=sm)

## Failure mode 2: Z is a bad proxy (no M->Z or D->Z)

Z has no direct relationship to D or M. So Z is not a good proxy treatment. However, X is a good proxy outcome and this can allow us to detect the failure mode. 

Unlike failure mode 1, Z is uncorrelated with D, which makes the dual assumption violated. Because the dual doesn't have a solution, the weakIV test is invalid, as the covariance calculation depends on the validity of the dual moment. We also expect the rank covariance test to fail as there is no relationship between Z and X. 

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has no direct relationship to M,
# X has direct relationship to M, X has no direct relationship to Y
d, e, f, g = 0.0, 0.0, 1.0, 0.0
sm = 2.0
run_summary(n, pz, px, a, b, c, d, e, f, g, sm)

## Failure Mode 3: X is a bad proxy (no M->X or X->Y)

Z is a good proxy, but X is not and is unrelated to M. This makes the existence of a solution to the primal IV assumption violated. We also expect the rank covariance test to fail as there is no relationship between Z and X. 

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has direct relationship to M,
# X has no direct relationship to M, X has no direct relationship to Y
d, e, f, g = 0.0, 1.0, 0.0, 0.0
sm = 2.0
run_summary(n, pz, px, a, b, c, d, e, f, g, sm)

## Failure mode 4: X and Z are completely unrelated


Z and X are unrelated to all other variables (both are bad proxies). We expect the covariance rank test to can catch this failure mode.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has no direct relationship to Z, Z has no direct relationship to M,
# X has no direct relationship to M, X has no direct relationship to Y (but doesn't affect results if it does; rank test will still catch the failure)
d, e, f, g = 0.0, 0.0, 0.0, 0.0
sm = 2.0
run_summary(n, pz, px, a, b, c, d, e, f, g, sm)

## Failure Mode 5: M ~= D.

X, Z are good proxies, but the mediator is super correlated with the treatment, which leads to lack of identification of the direct effect. The weakIV and idstrength tests will catch this case.

In [ ]:
np.random.seed(123)
# Indirect effect is a*b, direct effect is c
a, b, c = 1.0, 1.0, .5
# D has direct relationship to Z, Z has direct relationship to M,
# X has direct relationship to M, X has direct relationship to Y
d, e, f, g = 1.0, 1.0, 1.0, 1.0
sm = 0.05
run_summary(n, pz, px, a, b, c, d, e, f, g, sm)